# Descriptors Wrokgraph Calculation

To create a workgraph and run a calculation, you have to define some inputs as AiiDA data types and build a workgraph. 

First we can import the profile

In [ ]:
from aiida import load_profile
load_profile()

First of all we need a structure on which to perform the calculations. It will be a NaCl structure that we define using ASE, or alternatively one can choose one of the structures in the folder `Structures`.

The input structure in aiida-mlip needs to be saved as a StructureData type:

In [ ]:
from aiida.orm import StructureData
from ase.build import bulk
from ase.io import read

# structure = StructureData(ase=read("Structures/qmof-ffeef76.cif"))
structure = StructureData(ase=bulk("NaCl", "rocksalt", 5.63))

Then we need to choose a model and architecture to be used for the calculation and save it as ModelData type, a specific data type of this plugin.
In this example we use MACE with a model that we download from this URL: "https://github.com/stfc/janus-core/raw/main/tests/models/mace_mp_small.model", and we save the file in the cache folder (default="~/.cache/mlips/"):


In [ ]:
from aiida_mlip.data.model import ModelData

uri = "https://github.com/stfc/janus-core/raw/main/tests/models/mace_mp_small.model"
model = ModelData.from_uri(uri, architecture="mace_mp", cache_dir="mlips")

Another parameter that we need to define as AiiDA type is the code. Assuming the code is saved as `janus` in the `localhost` computer, the code info that are needed can be loaded as follow:

In [ ]:
from aiida.orm import load_code

code = load_code("janus@localhost")

The other inputs can be set up as AiiDA Str. There is a default for every input except the structure and code. This is a list of possible inputs:

In [ ]:
from aiida.orm import Bool, Str

inputs = {
    "code": load_code("janus@localhost"),
    "model": model,
    "struct": structure,
    "arch": Str(model.architecture),
    "precision": Str("float64"),
    "device": Str("cpu"),
    "metadata": {"options": {"resources": {"num_machines": 1}}},
    "invariants_only": Bool(True),
    "calc_per_atom": Bool(True),
    "calc_per_element": Bool(True),
}

We then load the calculation using `CalculationFactory` and the entrypoint, `"mlip.descriptors"` in this case:

In [ ]:
from aiida.plugins import CalculationFactory

DescriptorsCalc = CalculationFactory("mlip.descriptors")

Now we can start building the workgraph. First we use `graph_builder` to get the inputs and run the calculation

In [ ]:
from aiida_workgraph import WorkGraph, task
from aiida.engine import run_get_node, CalcJob, WorkChain

@task.graph_builder(outputs=[{"name" : "final_structure", "from": "ctx.final_structure"}])
def calculation(
    calc: CalcJob,
    calc_inputs: dict,
    ):

    wg = WorkGraph("Run Calculation")
    wg.add_input("workgraph.any", "calc_inputs")

    calc_task = wg.add_task(
        calc,
        name="run_calc",
        **calc_inputs
        )

    wg.update_ctx(
        {"final_structure": calc_task.outputs.xyz_output}
    )
    wg.outputs.final_structure = wg.ctx.final_structure

    return wg

Once the task has been created, we can then create a workgraph to pass in the inputs

In [ ]:
from aiida_workgraph import WorkGraph

wg = WorkGraph("Descriptors Calculation")
wg.add_input("workgraph.any", "calc_inputs")

wg.add_task(
    calculation, 
    calc = DescriptorsCalc,
    name="descriptors_calc",
    calc_inputs = inputs
    )

Now we can visualise the tasks

In [ ]:
wg

If everything has been setup correctly we can then run the workgraph, which runs the calculations with the inputs and get an output

In [ ]:
wg.run()

We can check if the graph has given us the correct output flle


In [ ]:
wg.tasks.descriptors_calc.outputs.final_structure

Finally we can visualise the WorkGraph

In [ ]:
from aiida_workgraph.utils import generate_node_graph

generate_node_graph(wg.pk)